# How random am I?
Just how good are people at generating random numbers?  We're going to try to find out here in this project. We're going to compare a series of user-generated random "coin toss" type events to actual coin toss events. In particular, we're going to see if people make random sequences or whether there's a bias embedded in there. We'll do this by looking at the distribution of run-lengths (how often do you have N in a row?).  For example, if 1=heads and 0=tails, the sequence `1 0 1 1 0 1 0 1 0 1 1 0` would have, for heads, 3 1-length runs, 2 2-length runs, and no 3-length runs.

## Getting the user input
For the first part of this project, you'll collect a series of 1's and 0's from a user. There are fancier ways of doing this but, trust me, when tossing in cross-platform aspects, macOS security, and simplicity, we're going to just use the `input` method in Python. The downside to this is that it just asks you to enter in a string and waits until you hit Enter. There's no ability to filter the keys and no ability to make sure exactly N valid keys have been pressed.

So, *write a function that has the user ostensibly press a bunch of 1's or 0's. Then, take that input, remove anything other than 1 or 0. Do this using a list comprehension. You can keep it as a list or convert it back to a string.* Once that basic bit is done, *set it up so that it can take a `min` (default 20) number of 1's and 0's that must be in there and enforce this.*

Remember, a string is something you can iterate over. Here, for example, taking `foo` and filtering out the numbers:


In [2]:
foo='Sharks are4 older1 than trees'
print(foo)
bar=[i for i in foo if not i.isnumeric()]
print(bar)
baz=''.join(i for i in foo if not i.isnumeric())
print(baz)

Sharks are4 older1 than trees
['S', 'h', 'a', 'r', 'k', 's', ' ', 'a', 'r', 'e', ' ', 'o', 'l', 'd', 'e', 'r', ' ', 't', 'h', 'a', 'n', ' ', 't', 'r', 'e', 'e', 's']
Sharks are older than trees


In [3]:
# ANSWER
!pip install icecream
from icecream import ic

def GetUserSequence(min=20):
    #inp='10102001010040130101030100339139101010110010101010'
    l=0
    while l<min:
        inp=input(f"Enter in at least {min} 1's and 0's")
        output=[k for k in inp if k in ['0','1']]
        l=len(output)
        if l<min:
            print(f"You need at least {min} items")
    return ''.join(output)

userseq=GetUserSequence()


Defaulting to user installation because normal site-packages is not writeable


## OK, so what's actually random?
Here, write come code that actually makes a random sequence of 0's and 1's of length n (default 1000), returning this as a string.  Later on, we'll use numpy and scipy, but Python now has decent random numbers built in.  Have a look at [`random.choices`](https://docs.python.org/3/library/random.html#random.choices).

In [7]:
import random
print(random.choices(['duck','go'],[10,2],k=10))


['duck', 'duck', 'duck', 'go', 'duck', 'go', 'duck', 'duck', 'go', 'duck']


In [34]:
# ANSWER
import random
def GenRandom(n=1000):
    return ''.join(random.choices(['0','1'],k=n))

rndseq=GenRandom()
#ic(rnd)
#print(userseq)
#rnd=GenRandom()
print(rndseq)

1000100100111010011101100101001101001011011101100111010100000011100010001001010111010011111010001100110100010101000110101011011111011011010100011100011110101101010011011100110001101011010110100010110111101110001110010000010001010111001000001010111111010011000101001010010010100101101010010011011000000010001011010010100100101010000010101101110100100111110101100110011101000001101011000100101100001000001001101010100110100110011011101100011111000110101101000010101111011000001010111101100011001110010011011010001010011101101110100011001101110011001100001001010111110000111010010100110101001111110011000100110001111011110010011111100111010011011000111010101101111110000011110100001011010001111011000100100111110111110010010100000101101010011011010001000011101010111101000001000011110101101110011100110110101101100001111101101101010001101001001010010110101010100011010001101001100011111000000111001011100000001101001011110010010110110011010101010110101010100110111000100011001100000011101001100111011010

# The fun part
Now comes the fun part.  We need to see just how often patterns come up. In particular, we're going to look for how often we get runs of length 1, runs of length 2, of length 3, ... length 8.  Python has a nice [`count`](https://docs.python.org/3/library/stdtypes.html?highlight=count#str.count) function that works on strings that we might think to use. But, the trouble is, this counts "non-overlapping occurrences of substrings".  Have a look at this sample.  We should end up with 1 run of length 1 and 1 of length 3, but none of length 2.

In [13]:
foo='01001110'
print(foo.count('1'))
print(foo.count('11'))
print(foo.count('111'))

4
1
1


Well that's not quite right... It found 4 of length=1, 1 of length=2 (*wait - think about why it came up with just 1 of these and not 2*), and 1 of length 3.

What if we made it look for 0's beforehand to make sure we're at the start of a run?

In [14]:
print(foo.count('01'))
print(foo.count('011'))
print(foo.count('0111'))

2
1
1


Closer, I suppose, but still not there.  What if we looked for the full start with 0, thing, and then end with 0?

In [15]:
print(foo.count('010'))
print(foo.count('0110'))
print(foo.count('01110'))

1
0
1


Hey, that looks good!  Let's just test it one more time though

In [28]:
foo='0101010101010101010101010'
print(foo.count('010'))
print(foo.count('0110'))
print(foo.count('01110'))

6
0
0


We were so close weren't we? I mean, really now.  6?  I count 12 in there.  *Why is it coming up with only 6?* Once we figure that out, *what might we do about it?*

Now, write a function that:

1. Takes in a string and figures out the number of run-lenghts of 1's from 1-8.  Remember, your string could start or end with a 1, so any solution you come up with has to handle this.
2. Divides those counts by the length of the string itself to, in some ways, normalize this so that short and long inputs are on roughly an even "odds of run-length X" kind of footing.  No, you can't use numpy.  A key point here is to think around obstacles for solutions.
3. Returns a list with those "probabilities" as a list

Then, run this on both your user-generated string and on the random string and give me a pretty printout of the results.

In [ ]:
def CalcRunLengthProbs(s):


In [32]:
#ANSWER

def CalcRunLengthProbs(s,return_counts=False):
    orig_l=len(s)
    s='0'+s+'0' # Prepend and append a 0 to our string to make sure we catch start / end of runs
    s=s.replace('0','00')
    counts=8*[0.0]
    counts[0]=s.count('010')
    counts[1]=s.count('0110')
    counts[2]=s.count('01110')
    counts[3]=s.count('011110')
    counts[4]=s.count('0111110')
    counts[5]=s.count('01111110')
    counts[6]=s.count('011111110')
    counts[7]=s.count('0111111110')
    probs=[x/orig_l for x in counts]
    if return_counts:
        return counts
    return probs


z='0101101010110001111'
p=CalcRunLengthProbs(z)
c=CalcRunLengthProbs(z,return_counts=True)
print(z)
print(p)
print(c)

0101101010110001111
[0.15789473684210525, 0.10526315789473684, 0.0, 0.05263157894736842, 0.0, 0.0, 0.0, 0.0]
[3, 2, 0, 1, 0, 0, 0, 0]


In [38]:
#ANSWER
p_user=CalcRunLengthProbs(userseq)
p_rnd=CalcRunLengthProbs(rndseq)
print('Len User  Random')
for i in range(len(p_user)):
    print(f"{i+1}   {p_user[i]:.2f}  {p_rnd[i]:.2f}")

Len User  Random
1   0.34  0.15
2   0.07  0.08
3   0.00  0.03
4   0.00  0.01
5   0.00  0.01
6   0.00  0.00
7   0.00  0.00
8   0.00  0.00
